In [1]:
from collections import defaultdict
import re
import pickle
import os
import random
import numpy as np
import re
from shutil import rmtree
import csv

In [2]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # source_blob_name = "storage-object-name"
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Blob {} downloaded to {}.".format(
            source_blob_name, destination_file_name
        )
    )

def format_category(category):
    name = ' '.join(category.strip().lower().split())
    name = name.replace("&", "and").replace('-', ' ').replace("'", '').replace(',', '').replace('/', '')
    return name

def format_text(text):
    ft = re.sub(r'[^\x00-\x7f]',r'', text.lower())
    return ft 
    
filename = '/home/sarahwooders_gmail_com/transformers/train.csv'


dataset = []
with open(filename, newline='',encoding = "utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dataset.append({
            'title': format_text(row['Title']),
            'description': format_text(row['Description']),
            'category': [format_category(row['Section']), format_category(row['Restaurant Category'])],
            'restaurant_category': format_category(row['Restaurant Category']),
            'section': format_category(row['Section'])
        })

## Shard doordash data by category

This is basically a manual map-reduce. First for each shard, we separate by the category name, then we gather entries across shards by category.

In [171]:
# probably makes sense to auto-place something that says "burger" in the name into burger group

labels= {
    "pizza": ["pizza"],
    "burger": ["burger", "hamburger", "cheeseburger"],
    "sandwich": ["sandwich"], 
    "sushi": ["sushi", "sashimi", "nigiri"], 
    "salad": ["salad"], 
    "soup": ["soup"]
}

other = [
    "wrap", 
    "fries",
    "pasta", 
    "bruschetta",
    "dumpling", 
    "wings",
    "curry",
    "rice", 
    "bread", 
    "pancake", 
    "dessert",
    "stirfry",
    "bowl",
    "stew", 
    "hotpot", 
    "platter", 
    "plate", 
    "beverage", 
    "drink", 
    "sauce", 
    "treats", 
    "calamari", 
    "sides", 
    "cake", 
    "quesadilla",
    "burrito", 
    "enchilada", 
    "boba"
]

healthy_restaurant_categories = ["healthy", "vegetarian", "vegan"]
unhealthy_restaurant_categories = ["desserts", "fast food", "comfort food", "alcohol", "barbecue"]

healthy_sections = ["salads", "vegetable", "veggie", "chopped salads", "vegetarian", "vegan", "wrap"]
unhealthy_sections = [
    "desserts", 
    "wings", 
    "sweets", 
    "ice cream", 
    "beer", 
    "belgian waffles", 
    "ribs",
    "pizza",
    "fries", 
    "bruschetta",
    "dumpling", 
    "curry",
    "rice", 
    "bread", 
    "pancake", 
    "dessert",
    "meat",
    "stew", 
    "hotpot", 
    "beverage", 
    "drink", 
    "sauce", 
    "treats", 
    "calamari", 
    "sides", 
    "cake", 
    "quesadilla",
    "burrito", 
    "enchilada", 
    "boba"
]


def match(category, labels):

    matches = 0
    match = 'None'
    
    if re.search(rf"\b(?=\w)and\b", category, re.IGNORECASE): # no combinations
        return False
        
    for label in labels:
        if re.search(rf"\b(?=\w){label}(s|es)?\b", category, re.IGNORECASE):
            return True
            
    return False
        
# quarter pounder 

shard_size = 1000000

def get_by_category(shard):
    #datafile = '/home/sarahwooders_gmail_com/doordash/doordash_{}.pkl'.format(shard)
    #with open(datafile, 'rb') as f:
    #    print("Opened {}".format(datafile))
    #    json = pickle.load(f)
    #    all_entries = []
    #    for entry in json:
    #        title = entry['title']
    #        section = entry['metadata']['section']
    #        description = entry['metadata']['description']
    #        category = entry['metadata'].get('restaurant_category', None)
    #        all_entries.append((title, description, section, category))
    
    all_entries = [(d['title'], d['description'], d['section'], d['restaurant_category']) \
          for d in dataset[shard*shard_size:(shard+1)*shard_size]]
    
    print("Loaded all entries", len(all_entries), shard*shard_size, (shard+1)*shard_size)
    category_to_entries = defaultdict(list)
    for entry in all_entries:
        og_category = entry[2]  + ' ' + entry[3]
        format_category = re.sub(r"\s+", '-', og_category)
        format_category = re.sub(r"\&", "and", format_category)
        format_category = format_category.lower()
        # (title, description) in a list
        
        format_category = None
        if og_category is None:
            continue
        if match(og_category, unhealthy_sections) or match(og_category, unhealthy_restaurant_categories):
            format_category = "unhealthy"
        if match(og_category, healthy_sections) or match(og_category, healthy_restaurant_categories):
            if format_category != None:
                continue
            format_category = "healthy"
            
        if format_category is None:
            continue
            
        if len(entry[0]) == 0:
            continue
            
        category_to_entries[format_category].append((entry[0], entry[1], entry[2], entry[3]))
        #category_to_entries[format_category].append((entry[0], entry[1]))
    print("Made dictionary")
    for category, entries in category_to_entries.items():
        print(category, len(entries))
        with open(
            '/home/sarahwooders_gmail_com/doordash-sharded/doordash_{}_{}'.format(category, shard), 'wb') as f:
            pickle.dump(entries, f)

In [172]:
match("veggie tray specialties", healthy_sections)

True

In [173]:
rootdir = '/home/sarahwooders_gmail_com/doordash-sharded'
targetdir = '/home/sarahwooders_gmail_com/doordash-by-dish-no-and'

if os.path.exists(rootdir):
    rmtree(rootdir)
    os.mkdir(rootdir)
    
if os.path.exists(targetdir):
    rmtree(targetdir)
    os.mkdir(targetdir)

for i in range(0, 16):
    get_by_category(i)

Loaded all entries 1000000 0 1000000
Made dictionary
unhealthy 277675
healthy 51895
Loaded all entries 1000000 1000000 2000000
Made dictionary
unhealthy 278055
healthy 51926
Loaded all entries 1000000 2000000 3000000
Made dictionary
unhealthy 277948
healthy 51902
Loaded all entries 1000000 3000000 4000000
Made dictionary
unhealthy 278471
healthy 51748
Loaded all entries 1000000 4000000 5000000
Made dictionary
unhealthy 277964
healthy 51835
Loaded all entries 1000000 5000000 6000000
Made dictionary
unhealthy 277401
healthy 51918
Loaded all entries 1000000 6000000 7000000
Made dictionary
unhealthy 278312
healthy 51611
Loaded all entries 1000000 7000000 8000000
Made dictionary
unhealthy 277068
healthy 51769
Loaded all entries 1000000 8000000 9000000
Made dictionary
unhealthy 278040
healthy 52232
Loaded all entries 1000000 9000000 10000000
Made dictionary
unhealthy 277500
healthy 51809
Loaded all entries 1000000 10000000 11000000
Made dictionary
unhealthy 278211
healthy 51631
Loaded all en

In [174]:
category_to_filenames = defaultdict(list)
rootdir = '/home/sarahwooders_gmail_com/doordash-sharded'
for filename in os.listdir(rootdir):
    _, category, shard = filename.split("_")
    category_to_filenames[category].append(
        os.path.join(rootdir, filename))

for category, filenames in category_to_filenames.items():
    all_items = []
    for filename in filenames:
        with open(filename, 'rb') as f:
            all_items.extend(pickle.load(f))
    with open('/home/sarahwooders_gmail_com/doordash-by-category/{}'.format(category), 'wb') as f:
        pickle.dump(all_items, f)

In [175]:
category_to_filenames = defaultdict(list)
    
for filename in os.listdir(rootdir):
    _, category, shard = filename.split("_")
    category_to_filenames[category].append(
        os.path.join(rootdir, filename))

for category, filenames in category_to_filenames.items():
    all_items = []
    for filename in filenames:
        with open(filename, 'rb') as f:
            all_items.extend(pickle.load(f))
    with open(targetdir + '/{}'.format(category), 'wb') as f:
        pickle.dump(all_items, f)

In [176]:
from collections import Counter
category_items_counter = Counter()
for filename in os.listdir(targetdir):
    with open(os.path.join(targetdir, filename),
              'rb') as f:
        all_items = pickle.load(f)
        category_items_counter[filename] = len(all_items)

In [177]:
print(category_items_counter.most_common())

# [('None', 10642937), ('other', 2415284), ('sandwich', 662065), ('pizza', 556996), ('salad', 500254), ('soup', 289421), ('sushi', 249551), ('burger', 235167)]

[('unhealthy', 4278858), ('healthy', 797390)]


# Take sharded data and clean

In [178]:
cuisine = 'unhealthy'
with open(os.path.join(targetdir, cuisine),
              'rb') as f:
    all_items = pickle.load(f)
    all_titles = [e[0] for e in all_items]

In [179]:
random.choices(all_items, k=10)

[('pork sides', '', 'breakfast sides', 'southern'),
 ('drink (22 oz.)', '', 'drinks', 'sandwiches'),
 ('mexican sodas', '', 'beverages', 'mexican'),
 ('squirt', '', 'drinks', 'seafood'),
 ('chicken fingers (5)', '', 'appetizers', 'pizza'),
 ('33. shrimp & pork rice noodle soup', '', 'rice noodle soups', 'asian'),
 ('whopper jr.',
  'our whopper jr. sandwich features one savory flame-grilled beef patty topped with juicy tomatoes, fresh lettuce, creamy mayonnaise, ketchup, crunchy pickles, and sliced white onions on a soft sesame seed bun.',
  'beef',
  'fast food'),
 ('keema naan',
  'leaved hand-made white bread with garlic.',
  'breads',
  'halal'),
 ('basket cheddar cheese curds',
  'wisconsin white cheddar cheese curds /\nbattered / southwestern ranch',
  'sides',
  'chicken'),
 ('butter naan',
  'plain flour bread freshly cooked with butter.',
  'breads',
  'indian')]

In [180]:
def check_valid(title):
    check = "".join(title.split())  # remove whitespace
    check = check.replace("-", "")  # remove -, allowed
    return check.isalpha()

In [181]:
def format_valid(title):
    title = " ".join(title.split())  # whitespace to " "
    title = title.rstrip().lstrip()
    title = title.replace("-", " ")
    return title

In [182]:
letter_items = [format_valid(x[0]) for x in all_items if check_valid(x[0])]

In [183]:
len([x for x in letter_items if x==" "])

0

In [184]:
len(letter_items), len(all_items)

(2993203, 4278858)

In [107]:
random.choices(letter_items, k=10)

['chocolate chip cookie',
 'iced brewed',
 'lb mashed potatoes',
 'house special fried rice',
 'bk caf coffee',
 'vegetarian sicilian pizza',
 'chai tea latte',
 'de carne asada',
 'double filet o fish',
 'bread basket']

In [120]:
targetdir

'/home/sarahwooders_gmail_com/doordash-by-dish-no-and'

## Take sharded data and tokenize

In [108]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    do_lower_case=True)

##### Demonstrate with particular cuisine

In [121]:
cuisine = 'unhealthy'
with open(os.path.join(targetdir, cuisine),
              'rb') as f:
    all_items = pickle.load(f)

In [122]:
titles = [e[0] for e in all_items]
print(titles[:100])

['burrito with mexican sausage', "single 'n cheese steakburger 'n fries", 'tiramisu', 'n11. tr  chanh', 'steamed broccoli', 'cheeseburger', 'sprite', 'mangoneada (grande)', 'chinotto', '#4. mixed gyro', 'meatball', 'mocha (23oz)', 'original chicken crispers', 'coffee', 'plain milk', 'bacon cheeseburger', 'plain naan', 'chicken primavera pizza', '23. hu tieu xa xiu', 'sausage egg cheese mcgriddle meal', 'kids fish sticks & fries', 'california turkey club', 'italian cold cut (8")', 'fr5. egg roll fried rice plate', 'fresh brewed iced tea', 'chicken cordon bleu', 'margherita', 'chocolate on the ritz', '#18 3 little piggies mac (regular size)', '6 croquettes de poulet / 6 chicken croquettes', '2. pork noodle in soup (coke)', 'funnel cake', 'mole enchilada poblano', 'the alamo burrito', 'mushrooms stuffed breadsticks', 'gallon of mango iced tea', "cheryl's fudge brownie (each)", 'spaghetti', '28" al metro pizza', 'eggplant & ricotta', 'wings combo (6)', 'traditional lemonade', 'bottled drin

In [123]:
max_seq_length=64
tokenized_output = tokenizer.batch_encode_plus(
    titles[:5], max_seq_length=max_seq_length,
    pad_to_max_length=False)
print(tokenized_output)

{'input_ids': [[101, 11499, 24874, 10340, 10169, 10911, 53845, 10115, 11731, 10466, 10525, 102], [101, 11376, 112, 182, 10262, 13565, 28780, 43213, 59692, 112, 182, 35350, 10171, 102], [101, 102770, 15240, 10138, 102], [101, 182, 37115, 119, 32221, 62628, 10237, 102], [101, 45833, 10336, 93742, 20493, 10390, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


###### Getting max seq length

In [112]:
cuisine_to_lens = {}
for cuisine in os.listdir('/home/doordash-by-category'):
    if os.path.isdir('/home/doordash-by-category/{}'.format(cuisine)):
        continue
    print(cuisine)
    with open(os.path.join('/home/doordash-by-category', cuisine),
                  'rb') as f:
        all_items = pickle.load(f)
    titles = [e[0] for e in all_items]
    random.shuffle(titles)
    tokenized_output = tokenizer.batch_encode_plus(
        titles[:1000], max_seq_length=128,
        pad_to_max_length=False)
    lens = [len(input_ids) for input_ids in tokenized_output['input_ids']]
    cuisine_to_lens[cuisine] = lens

sandwiches
deals
ethiopian
pickup
soup
smoothie
good-for-groups
alcohol-stores
beauty
exclusive
tapas
thai
german
cafes
pho
alcohol
belgian
indian
mediterranean
seafood
vietnamese
catering
mexican
greek
comfort-food
irish
european
ramen
heritage
bakery
newly-added
french
pakistani
takeout
desserts
halal
chicken
bubble-tea
drinks
kwb
fondue
argentine
steak
pasta
cheese
italian
late-night
japanese
south-asian
chinese
peruvian
asian
breakfast
coffee-and-tea
southern
healthy
other
cajun
brazilian
lunch
russian
none
pizza
delis
korean
barbecue
vegetarian
grocery
caribbean
noodles
filipino
british
gastropubs
burmese
sushi
middle-eastern
vegan
blank
latin-american
convenience
poke
poutineries
coffee
kosher
convenience-exclusive
gluten-free
african
burgers
salad
fast-food
food-and-drink
american
australian
spanish


In [28]:
{c: max(l) for c, l in cuisine_to_lens.items()}

{'sandwiches': 34,
 'deals': 26,
 'ethiopian': 27,
 'pickup': 27,
 'soup': 39,
 'smoothie': 44,
 'good-for-groups': 22,
 'alcohol-stores': 25,
 'beauty': 20,
 'exclusive': 10,
 'tapas': 37,
 'thai': 24,
 'german': 37,
 'cafes': 29,
 'pho': 53,
 'alcohol': 26,
 'belgian': 24,
 'indian': 27,
 'mediterranean': 21,
 'seafood': 26,
 'vietnamese': 43,
 'catering': 31,
 'mexican': 27,
 'greek': 37,
 'comfort-food': 30,
 'irish': 25,
 'european': 27,
 'ramen': 22,
 'heritage': 17,
 'bakery': 28,
 'newly-added': 19,
 'french': 36,
 'pakistani': 22,
 'takeout': 20,
 'desserts': 27,
 'halal': 28,
 'chicken': 24,
 'bubble-tea': 34,
 'drinks': 33,
 'kwb': 26,
 'fondue': 24,
 'argentine': 30,
 'steak': 26,
 'pasta': 33,
 'cheese': 29,
 'italian': 30,
 'late-night': 24,
 'japanese': 35,
 'south-asian': 28,
 'chinese': 29,
 'peruvian': 39,
 'asian': 38,
 'breakfast': 27,
 'coffee-and-tea': 25,
 'southern': 25,
 'healthy': 34,
 'other': 25,
 'cajun': 29,
 'brazilian': 31,
 'lunch': 18,
 'russian': 22,


### Saving tokenization

In [124]:
#rootdir = '/home/doordash-by-category/cleaned-tokenized/bert-base-multilingual-cased'
rootdir = '/home/sarahwooders_gmail_com/transformers/doordash-tmp'
max_seq_length = 64
assert os.path.exists(rootdir)

In [125]:
file_pattern = "titles_{cuisine}_{max_seq_len}_{input_key}"

In [126]:
do_clean = True

def format_valid(title):
    title = " ".join(title.split())  # whitespace to " "
    title = title.rstrip().lstrip()
    title = title.replace("-", " ")
    return title

def check_valid(title):
    check = "".join(title.split())  # remove whitespace
    check = check.replace("-", "")  # remove -, allowed
    return check.isalpha()

In [127]:
targets_to_categories = {
    # Label to list of filenames corresponding to that label
    'mexican': ['mexican'],
    'chinese': ['chinese'],
    'american': ['american', 'burgers'],
    'italian': ['italian', 'pasta'],
    'thai': ['thai'],
    'indian': ['indian'],
    'japanese': ['japanese', 'ramen', 'sushi'],
    'other': ['african', 'alcohol', 'argentine', 'australian',
             'bakery', 'belgian', 'brazilian', 'burmese', 'desserts',
             'drinks', 'ethiopian', 'filipino', 'french',
             'german', 'greek', 'korean', 'vietnamese', 'poke']
}
#all_categories = [item for sublist in targets_to_categories.values() for item in sublist]
all_categories = ['healthy', 'unhealthy']

In [128]:
print(max_seq_length)

64


In [129]:
file_pattern = "titles_{cuisine}_{max_seq_len}_{input_key}"
lens = {}
for cuisine in all_categories:# os.listdir('/home/doordash-by-category'):
    if os.path.isdir('/home/sarahwooders_gmail_com/doordash-by-category/{}'.format(cuisine)):
        # Not a real cuisine, for example bert-base-multilingual-cased is a folder containing tokenizations
        continue
    with open(os.path.join('/home/sarahwooders_gmail_com/doordash-by-category', cuisine), 'rb') as f:
        all_items = pickle.load(f)
    titles = [e[0] for e in all_items]
    if do_clean:
        titles = [format_valid(t) for t in titles if check_valid(t)]
    
    lens[cuisine] = len(titles)
    if os.path.exists(os.path.join(
        rootdir, file_pattern.format(
            cuisine=cuisine, max_seq_len=max_seq_length, 
            input_key="input_ids"))):
        # Tokenziation already exists in the given folder
        continue

    print(cuisine, len(titles))
    tokenized_output = tokenizer.batch_encode_plus(
        titles,
        max_length=max_seq_length,
        pad_to_max_length=True)
    
    print("Done tokenizing")
    for input_key, arr in tokenized_output.items():
        filename = os.path.join(rootdir,
                                file_pattern.format(cuisine=cuisine,
                                      max_seq_len=max_seq_length,
                                      input_key=input_key))
        arr = np.array(arr)
        fp = np.memmap(filename, dtype='int64', mode='w+',
                      shape=arr.shape)
        fp[:] = arr[:]
        del fp

unhealthy 583485
Done tokenizing


In [132]:
with open(os.path.join(rootdir, 'lens.pkl'), 'wb') as f:
    # Keep lens which is needed for loading a memmap
    pickle.dump(lens, f)

In [133]:
print(lens.keys())

dict_keys(['healthy', 'unhealthy'])


In [135]:
lens

{'healthy': 112415, 'unhealthy': 583485}

In [134]:
import wandb

dataset_name = 'doordash'
    
# Store dataset on W&B
artifact = wandb.Artifact(dataset_name, type='dataset')
artifact.add_dir(rootdir)
run = wandb.init(project="bert-classification", job_type='preprocessing')
print(run.log_artifact(artifact))

wandb: Adding directory to artifact (/home/sarahwooders_gmail_com/transformers/doordash-tmp)... Done. 0.8s


None


In [57]:
artifact = run.use_artifact('doordash:latest', type='dataset')
artifact_dir = artifact.download()
print(artifact_dir)

wandb: Downloading large artifact doordash:latest, 6156.14MB. 88 files... Done. 3.5s


/home/sarahwooders_gmail_com/.cache/wandb/artifacts/final/dataset/864bac0c35a47e10b0be48b3fc609aa2/artifact


In [58]:
rootdir

'/home/doordash-by-category/cleaned-tokenized/bert-base-multilingual-cased'

# Dataloader

In [147]:
import torch
import itertools
from collections import Counter, OrderedDict, defaultdict
from torch.utils.data import DataLoader

In [148]:
class IterableTitles(torch.utils.data.IterableDataset):
    def __init__(self, cuisine, root_dir, max_seq_length, len_dict):
        super(IterableTitles).__init__()
        self.cuisine = cuisine
        self.max_seq_length = max_seq_length
        self.datafile_pattern = os.path.join(
            root_dir, 
            'titles_{}_{}_{{}}'.format(cuisine, max_seq_length))
        self.shape = (len_dict[cuisine], max_seq_length)
        self.length = len_dict[cuisine]

    def __len__(self):
        return self.length

    def __iter__(self):
        input_key_to_memmap = {}
        for input_key in ['input_ids',
                          'attention_mask',
                         'token_type_ids']:
            datafile = self.datafile_pattern.format(input_key)
            fp = np.memmap(
                datafile, dtype='int64', mode='r+',
                shape=self.shape)
            input_key_to_memmap[input_key] = fp
        while True:
            i = random.choice(range(self.shape[0]))
        #for i in itertools.cycle(range(self.shape[0])):
            yield (
                input_key_to_memmap['input_ids'][i],
                input_key_to_memmap['token_type_ids'][i],
                input_key_to_memmap['attention_mask'][i])


class MultiStreamDataLoader:
    def __init__(self, targets_to_categories, batch_size, rootdir):
        self.targets = list(sorted(targets_to_categories.keys()))
        self.targets_to_categories = targets_to_categories
        
        self.categories_to_target = OrderedDict()
        self.targets_to_categories_weights= {}
        self.categories_to_dataset = {}
        self.categories_to_dataset_iter = {}
        with open(os.path.join(rootdir, 'lens.pkl'), 'rb') as f:
            self.categories_to_len = pickle.load(f)
        
        for t, list_of_c in targets_to_categories.items():
            for c in list_of_c:
                self.categories_to_target[c] = t
                dataset = IterableTitles(c, rootdir, 64, self.categories_to_len)
                self.categories_to_dataset[c] = dataset
                self.categories_to_dataset_iter[c] = iter(DataLoader(dataset, batch_size=None))

        for t, c_list in self.targets_to_categories.items():
            self.targets_to_categories_weights[t] = [self.categories_to_len[c] for c in c_list]

        self.batch_size = batch_size
        self.total_samples = sum(self.categories_to_len.values())


    def __len__(self):
        return self.total_samples//self.batch_size
    

    def __iter__(self):
        while True:
            buffer = []
            labels = []
            target_choices = random.choices(self.targets, k=self.batch_size)
            category_choices = []
            for t in target_choices:
                category_choices.append(random.choices(
                    self.targets_to_categories[t],
                    weights=self.targets_to_categories_weights[t],
                    k=1)[0])

            category_counter = Counter(category_choices)
            category_labels = []
            for category, num_sample in category_counter.items():
                category_labels.extend([category for _ in range(num_sample)])
                l_num = self.targets.index(self.categories_to_target[category])
                labels.extend([l_num for _ in range(num_sample)])
                buffer.extend(
                    [next(self.categories_to_dataset_iter[category]) for _ in range(num_sample)]
                )
            yield (torch.stack([b[0] for b in buffer]),
                   torch.stack([b[1] for b in buffer]),
                   torch.stack([b[2] for b in buffer]),
                   torch.tensor(labels),
                   category_labels
                  )

In [149]:
rootdir

'/home/sarahwooders_gmail_com/transformers/doordash-tmp'

In [154]:
targets_to_categories = {
    # Label to list of filenames corresponding to that label
    'mexican': ['mexican'],
    'chinese': ['chinese'],
    'american': ['american', 'burgers'],
    'italian': ['italian', 'pasta'],
    'thai': ['thai'],
    'indian': ['indian'],
    'japanese': ['japanese', 'ramen', 'sushi'],
    'other': ['african', 'argentine', 'australian',
             'bakery', 'belgian', 'brazilian', 'burmese', # 'desserts',
             'drinks', 'ethiopian', 'filipino', 'french', # 'alcohol'
             'german', 'greek', 'korean', 'vietnamese', 'poke']
}

targets_to_categories = {
    "healthy": ["healthy"], 
    "unhealthy": ["unhealthy"]
}

ds = MultiStreamDataLoader(targets_to_categories,
                           batch_size=100,
                           rootdir=rootdir)

In [155]:
ds.targets

['healthy', 'unhealthy']

In [156]:
ds.targets_to_categories_weights

{'healthy': [112415], 'unhealthy': [583485]}

In [157]:
[(t, sum(w)) for t, w in ds.targets_to_categories_weights.items()]

[('healthy', 112415), ('unhealthy', 583485)]

In [167]:
for batch_num, batch in enumerate(ds):
    if batch_num == 5:
        break
    print(Counter(batch[3].numpy()))
    input_ids = batch[0]
    labels = batch[3]
    categories = batch[4]
    for i, ids in enumerate(input_ids):
        text = tokenizer.convert_tokens_to_string(
                tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=True))
        print(
            ds.targets[labels[i]], "({})".format(categories[i]),
            ":",
            text,
            # "\n",
            # ids
        )
        test_input = tokenizer.batch_encode_plus(
            [(text, None)],
            max_length=max_seq_length,
            pad_to_max_length=True,
        )

        model_inp = dict(test_input)
        model_inp['labels'] = [0]
        model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
        model_out = model(**model_inp)

        predicted_logits = model_out[1].detach().cpu().numpy()
        predicted_class = np.argmax(predicted_logits)
        print(text)
        print(['healthy', 'unhealthy'][predicted_class], predicted_logits)
        print()
        print("=====")

Counter({1: 54, 0: 46})
healthy (healthy) : granny apple turkey executive box
granny apple turkey executive box
healthy [[ 3.8637886 -3.0192711]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : cobb salad
cobb salad
healthy [[ 2.800048  -2.2164183]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : fish and chips
fish and chips
unhealthy [[-0.27743867  0.4421656 ]]

=====
healthy (healthy) : pepsi
pepsi
healthy [[0.17363457 0.05254225]]

=====
healthy (he

unhealthy (unhealthy) : roasted duck curry
roasted duck curry
unhealthy [[-3.779735   3.0018094]]

=====
unhealthy (unhealthy) : lemonade
lemonade
unhealthy [[-0.88010997  0.9266181 ]]

=====
unhealthy (unhealthy) : nariwal naan
nariwal naan
unhealthy [[-3.8380103  3.0844285]]

=====
unhealthy (unhealthy) : water
water
unhealthy [[-1.113453  1.097355]]

=====
unhealthy (unhealthy) : coke zero
coke zero
unhealthy [[-0.40364355  0.5269718 ]]

=====
unhealthy (unhealthy) : gobble gobble burger
gobble gobble burger
unhealthy [[-0.34879443  0.5025656 ]]

=====
unhealthy (unhealthy) : cappuccino
cappuccino
unhealthy [[-0.6795612  0.7704322]]

=====
unhealthy (unhealthy) : housemade breakfast sausage
housemade breakfast sausage
unhealthy [[-1.4806468  1.3675679]]

=====
unhealthy (unhealthy) : blueberry cheese knish
blueberry cheese knish
unhealthy [[-0.9485113   0.98681575]]

=====
unhealthy (unhealthy) : jungle curry
jungle curry
unhealthy [[-0.83027244  0.8850155 ]]

=====
unhealthy (unhea


healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : taro smoothies
taro smoothies
healthy [[ 0.52146775 -0.23618375]]

=====
healthy (healthy) : pitaya bowls
pitaya bowls
healthy [[ 2.5030334 -1.966343 ]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : crave no more smoothie
crave no more smoothie
healthy [[ 3.4471838 -2.7118409]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : water
water
unhealthy [[-1.113453  1.097355]]

=====
healthy (healthy) : smoked sausage
smoked sausage
unhealthy [[-0.6665275  0.

healthy (healthy) : nestea
nestea
healthy [[ 0.43663317 -0.17350656]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : fried smoked gouda grit balls
fried smoked gouda grit balls
healthy [[ 3.2448463 -2.5352945]]

=====
healthy (healthy) : asian crunch wrap
asian crunch wrap
healthy [[ 3.8770218 -3.0087593]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : thin mint
thin mint
healthy [[ 2.275963  -1.7929192]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.4100

healthy (healthy) : raspberry romance
raspberry romance
healthy [[ 3.5843394 -2.8351903]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : chicken veggie bowl
chicken veggie bowl
healthy [[ 3.6474793 -2.8340979]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : cinnamon toast crunch
cinnamon toast crunch
healthy [[ 0.5035244 -0.268469 ]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : tunacado
tunacado
healthy [[ 3.9646106 -3.0988724]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : 

healthy [[ 0.410018   -0.12013413]]

=====
healthy (healthy) : side of chick

In [145]:
print('hello')

hello


In [146]:
all_categories

['healthy', 'unhealthy']

# Evaluate Batch 

In [161]:
import transformers
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval, BertForSequenceClassification)
import torch
import numpy as np

In [166]:
ckpt_dir = "/home/transformers-public/output_dirs/sarah-test4/checkpoint-6000"
config = AutoConfig.from_pretrained(ckpt_dir)
tokenizer = BertTokenizer.from_pretrained(
    ckpt_dir,
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    ckpt_dir,
    from_tf=False,
    config=config,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

labels = ['healthy', 'unhealthy']

In [ ]:
labels = ['healthy', 'unhealthy']

counts = [0, 0]
for p in dataset:
    text = p['title'].lower() 
    test_input = tokenizer.batch_encode_plus(
        [(text, None)],
        max_length=max_seq_length,
        pad_to_max_length=True,
    )
    
    model_inp = dict(test_input)
    model_inp['labels'] = [0]
    model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
    model_out = model(**model_inp)

    predicted_logits = model_out[1].detach().cpu().numpy()
    predicted_class = np.argmax(predicted_logits)
    print(text)
    print(labels[predicted_class], predicted_logits)
    counts[predicted_class] += 1
    print()